In [1]:
import pandas as pd
import requests
import json
import ast
from bs4 import BeautifulSoup

In [2]:
restaurant_data_df = pd.read_excel("rw_restaurants_2025-07_v4.xlsx")
restaurant_data_df

,borough,neighborhood,primaryCategory,primaryLocation,ctaButtons,tags,shortTitle,slug,image,summary,website,menuFileUrl,collections,mealTypes,restaurantInclusionWeek,ecommerce,rest_week_site_url,image_url
0,Manhattan,Midtown East,Restaurants,Midtown East,"{'primary': {'label': {'presale': False, 'gene...","['Italian', 'Midtown East']",Fresco by Scotto,fresco-by-scotto,{'url': '//images.ctfassets.net/1aemqu6a6t65/J...,"For more than three decades, Fresco by Scotto,...",https://www.frescobyscottony.com,NaN,['celebrity-chefs'],"['$45 Lunch Price', '$60 Dinner Price']","['Week 3 (Aug 4 - Aug 10)', 'Week 4 (Aug 11 - ...",NaN,https://www.nyctourism.com/restaurant-week/fre...,https://images.ctfassets.net/1aemqu6a6t65/JRA3...
1,Manhattan,Lower East Side,Restaurants,Lower East Side,"{'primary': {'label': {'presale': False, 'gene...","['Korean', 'Lower East Side']",Sae Ron,sae-ron,{'url': '//images.ctfassets.net/1aemqu6a6t65/1...,Sae Ron is a modern Korean kitchen dedicated t...,https://www.saeronnyc.com,NaN,[],['$45 Dinner Price'],"['Week 2 (July 28 - Aug 3)', 'Week 3 (Aug 4 - ...",NaN,https://www.nyctourism.com/restaurant-week/sae...,https://images.ctfassets.net/1aemqu6a6t65/1G7R...
2,Manhattan,Midtown West,Restaurants,Midtown West,"{'primary': {'label': {'presale': False, 'gene...","['American (New)', 'Midtown West']",Redeye Grill,redeye-grill,{'url': '//images.ctfassets.net/1aemqu6a6t65/7...,This cavernous restaurant captures the richnes...,https://www.redeyegrill.com,NaN,[],"['$30 Lunch Price', '$60 Dinner Price', '$60 S...","['Week 2 (July 28 - Aug 3)', 'Week 3 (Aug 4 - ...","{'title': 'OpenTable', 'partnerName': 'OpenTab...",https://www.nyctourism.com/restaurant-week/red...,https://images.ctfassets.net/1aemqu6a6t65/79jn...
3,Manhattan,Flatiron District,Restaurants,Flatiron District,"{'primary': {'label': {'presale': False, 'gene...","['Italian', 'Flatiron District']",Zero Otto Nove - Flatiron,zero-otto-nove,{'url': '//images.ctfassets.net/1aemqu6a6t65/5...,Roberto Paciullo's Flatiron District restauran...,https://zeroottonove.com/nyc,NaN,[],"['$45 Lunch Price', '$45 Dinner Price', '$45 S...","['Week 2 (July 28 - Aug 3)', 'Week 3 (Aug 4 - ...","{'title': 'OpenTable', 'partnerName': 'OpenTab...",https://www.nyctourism.com/restaurant-week/zer...,https://images.ctfassets.net/1aemqu6a6t65/54rW...
4,Manhattan,Hudson Square,Nightlife,Hudson Square,"{'primary': {'label': {'presale': False, 'gene...","['American (New)', 'Hudson Square']",Torch & Crown Brewing Company,torch-crown-brewing-company,{'url': '//images.ctfassets.net/1aemqu6a6t65/3...,"Torch & Crown Brewing Company, in Hudson Squar...",https://www.torchandcrown.com,NaN,[],"['$30 Lunch Price', '$45 Dinner Price', '$30 S...","['Week 2 (July 28 - Aug 3)', 'Week 3 (Aug 4 - ...",NaN,https://www.nyctourism.com/restaurant-week/tor...,https://images.ctfassets.net/1aemqu6a6t65/3WPZ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
622,Manhattan,Upper East Side,Restaurants,Upper East Side,"{'primary': {'label': {'presale': False, 'gene...","['Steakhouse', 'Upper East Side']",Catch n' Chop,catch-n-chop,"{'url': '', 'alt': ''}","Catch n' Chop, situated on the Upper East Side...",https://catchnchop.com,NaN,[],"['$60 Dinner Price', '$30 Sunday Lunch/Brunch ...","['Week 2 (July 28 - Aug 3)', 'Week 3 (Aug 4 - ...","{'title': 'OpenTable', 'partnerName': 'OpenTab...",https://www.nyctourism.com/restaurant-week/cat...,NaN
623,Manhattan,Upper West Side,Restaurants,Upper West Side,"{'primary': {'label': {'presale': False, 'gene...","['French', 'Upper West Side']",Amélie - Upper West Side,amelie-upper-west-side,"{'url': '', 'alt': ''}","""La Petite Amélie"" is smaller and more intimat...",https://www.amelieupperwest.com,NaN,[],"['$45 Dinner Price', '$45 Sunday Dinner Price']","['Week 2 (July 28 - Aug 3)', 'Week 3 (Aug 4 - ...",NaN,https://www.nyctourism.com/restaurant-week/ame...,NaN
624,Brooklyn,Downtown Brooklyn,Restaurants,Downtown Brooklyn,"{'primary': {'label': {'presa

In [3]:
def get_page_data(rest_week_site_url):
    page_data = {}
    req = requests.get(rest_week_site_url)
    if req.status_code != 200:
        return {"location_coords": {"lat": "", "lon": ""}, "pdf_url": "", "venueAddress": ""}
        
    soup = BeautifulSoup(req.text)
    next_data = soup.select("#__NEXT_DATA__")[0]
    next_data_json = json.loads(next_data.text)
    fields = next_data_json["props"]["pageProps"]["pageFields"]["fields"]
    page_data["location_coords"] = fields["location"]
    page_data["pdf_url"] = next_data_json["props"]["pageProps"]["page"].get("fileUrl", "")
    page_data["venueAddress"] = fields["venueAddress"]

    return page_data

In [4]:
def get_all_rest_data(search_data):
    page_data = get_page_data(search_data["rest_week_site_url"])
    
    rest_data = {}
    rest_data["name"] = search_data["shortTitle"]
    rest_data["pdf_url"] = page_data["pdf_url"]
    rest_data["slug"] = search_data["slug"]
    rest_data["url"] = search_data["rest_week_site_url"]
    rest_data["venueAddress"] = page_data["venueAddress"]
    rest_data["latitude"] = page_data["location_coords"]["lat"]
    rest_data["longitude"] = page_data["location_coords"]["lon"]
    rest_data["summary"] = search_data["summary"]
    rest_data["website"] = search_data["website"]
    rest_data["collections"] = search_data["collections"]
    rest_data["borough"] = search_data["borough"]
    rest_data["neighborhood"] = search_data["neighborhood"]
    rest_data["primaryCategory"] = search_data["primaryCategory"]
    rest_data["primaryLocation"] = search_data["primaryLocation"]
    rest_data["restaurantInclusionWeek"] = search_data["restaurantInclusionWeek"]
    rest_data["image_url"] = search_data["image_url"]
    
    mealTypes = ast.literal_eval(search_data["mealTypes"])
    rest_data["meal_type"] = "; ".join(mealTypes).replace(" Price", "")

    rest_tags = ast.literal_eval(search_data["tags"].replace("None", '""'))
    rest_data["tags"] = "; ".join(rest_tags).strip("; ")

    return rest_data

In [5]:
%%time
count = 1
total = len(restaurant_data_df)
all_rest_data = []
for index, restaurant in restaurant_data_df.iterrows():
    print(f"{count} out of {total}: {restaurant['shortTitle']}")
    rest_data = get_all_rest_data(restaurant)
    all_rest_data.append(rest_data)
    count += 1

1 out of 627: Fresco by Scotto
2 out of 627: Sae Ron
3 out of 627: Redeye Grill
4 out of 627: Zero Otto Nove - Flatiron
5 out of 627: Torch & Crown Brewing Company
6 out of 627: The Orchard Townhouse
7 out of 627: Sei Less
8 out of 627: Tha Phraya
9 out of 627: Xolo
10 out of 627: Strip House Midtown
11 out of 627: Evalyn’s Tap House
12 out of 627: Essex
13 out of 627: Baar Baar
14 out of 627: Ci Siamo
15 out of 627: Bobo
16 out of 627: Gyu-Kaku Japanese BBQ - Amsterdam
17 out of 627: Addictive NYC
18 out of 627: Sereneco
19 out of 627: Rowland's Bar & Grill
20 out of 627: Oceana
21 out of 627: Masseria East
22 out of 627: Momoya - Upper West Side
23 out of 627: Shan
24 out of 627: DiWine
25 out of 627: Hutong
26 out of 627: Tamarind Tribeca
27 out of 627: The Ribbon—Upper West Side
28 out of 627: Jack & Charlie’s No. 118
29 out of 627: RA Sushi Bar Restaurant – Times Square
30 out of 627: Veselka
31 out of 627: Wagamama - Murray Hill
32 out of 627: Hav & Mar
33 out of 627: Gyu-Kaku Ja

256 out of 627: Del Frisco's Grille - Brookfield Place
257 out of 627: Treadwell Park - Upper East Side
258 out of 627: Kraam
259 out of 627: Mito - Bayside
260 out of 627: Haven Rooftop
261 out of 627: Nick + Stef's Steakhouse
262 out of 627: Carmine's Times Square
263 out of 627: Porter House
264 out of 627: Molyvos
265 out of 627: Sant Ambroeus—Brookfield
266 out of 627: Leuca
267 out of 627: Ai Fiori
268 out of 627: No Work Allowed
269 out of 627: Atla
270 out of 627: House of the Red Pearl
271 out of 627: Meet the Meat
272 out of 627: Fish Cheeks
273 out of 627: Greywind
274 out of 627: Benjamin Prime
275 out of 627: Via 13
276 out of 627: Koloman
277 out of 627: Maiella
278 out of 627: Tin Marín
279 out of 627: The Shell Seafood Kitchen
280 out of 627: Siren Oyster Bar & Restaurant
281 out of 627: Sushi Lab Rooftop
282 out of 627: The Quimby
283 out of 627: El Coco - Chelsea
284 out of 627: Rosemary's East
285 out of 627: Cebu Bar & Bistro
286 out of 627: Ma•dé
287 out of 627: A

510 out of 627: Calle Dão - Bryant Park
511 out of 627: Grand Brasserie
512 out of 627: Burger & Lobster Bryant Park
513 out of 627: Brooklyn Bowl
514 out of 627: Carmine's - Upper West Side
515 out of 627: Jade Eatery & Lounge
516 out of 627: Psaraki
517 out of 627: The Dining Room at Gramercy Tavern
518 out of 627: Felice 56
519 out of 627: Longo Bros.
520 out of 627: Empellón
521 out of 627: Aromi
522 out of 627: Maison Pickle
523 out of 627: Le Rivage
524 out of 627: Boucherie Union Square
525 out of 627: Tasca NYC
526 out of 627: Takumen
527 out of 627: Marsanne
528 out of 627: Mas Tortilla
529 out of 627: Pylos
530 out of 627: Barawine
531 out of 627: Safari
532 out of 627: The Commissary at Metrograph
533 out of 627: Wayward Fare
534 out of 627: Smyth Tavern
535 out of 627: Margaux by La Sirène
536 out of 627: Mastro's Steakhouse NYC
537 out of 627: Mamazul
538 out of 627: Cha Cha Tang
539 out of 627: Pepolino
540 out of 627: Cafe Zaffri
541 out of 627: 8282
542 out of 627: Rava

In [6]:
all_rest_data_df = pd.DataFrame(all_rest_data)
all_rest_data_df

,name,pdf_url,slug,url,venueAddress,latitude,longitude,summary,website,collections,borough,neighborhood,primaryCategory,primaryLocation,restaurantInclusionWeek,image_url,meal_type,tags
0,Fresco by Scotto,https://nyc-tourism-public.s3.amazonaws.com/sr...,fresco-by-scotto,https://www.nyctourism.com/restaurant-week/fre...,"34 E. 52nd St.,New York,10022,NY",40.758816,-73.974514,"For more than three decades, Fresco by Scotto,...",https://www.frescobyscottony.com,['celebrity-chefs'],Manhattan,Midtown East,Restaurants,Midtown East,"['Week 3 (Aug 4 - Aug 10)', 'Week 4 (Aug 11 - ...",https://images.ctfassets.net/1aemqu6a6t65/JRA3...,$45 Lunch; $60 Dinner,Italian; Midtown East
1,Sae Ron,https://nyc-tourism-public.s3.amazonaws.com/sr...,sae-ron,https://www.nyctourism.com/restaurant-week/sae...,"99 Stanton St.,Manhattan,10002,NY",40.721162,-73.988116,Sae Ron is a modern Korean kitchen dedicated t...,https://www.saeronnyc.com,[],Manhattan,Lower East Side,Restaurants,Lower East Side,"['Week 2 (July 28 - Aug 3)', 'Week 3 (Aug 4 - ...",https://images.ctfassets.net/1aemqu6a6t65/1G7R...,$45 Dinner,Korean; Lower East Side
2,Redeye Grill,https://nyc-tourism-public.s3.amazonaws.com/sr...,redeye-grill,https://www.nyctourism.com/restaurant-week/red...,"890 Seventh Ave.,Manhattan,10019,NY",40.765194,-73.980684,This cavernous restaurant captures the richnes...,https://www.redeyegrill.com,[],Manhattan,Midtown West,Restaurants,Midtown West,"['Week 2 (July 28 - Aug 3)', 'Week 3 (Aug 4 - ...",https://images.ctfassets.net/1aemqu6a6t65/79jn...,$30 Lunch; $60 Dinner; $60 Sunday Dinner,American (New); Midtown West
3,Zero Otto Nove - Flatiron,,zero-otto-nove,https://www.nyctourism.com/restaurant-week/zer...,"15 W. 21st St.,Manhattan,10010,NY",40.740838,-73.991568,Roberto Paciullo's Flatiron District restauran...,https://zeroottonove.com/nyc,[],Manhattan,Flatiron District,Restaurants,Flatiron District,"['Week 2 (July 28 - Aug 3)', 'Week 3 (Aug 4 - ...",https://images.ctfassets.net/1aemqu6a6t65/54rW...,$45 Lunch; $45 Dinner; $45 Sunday Dinner,Italian; Flatiron District
4,Torch & Crown Brewing Company,https://nyc-tourism-public.s3.amazonaws.com/sr...,torch-crown-brewing-company,https://www.nyctourism.com/restaurant-week/tor...,"12 Vandam St.,Manhattan,10013,NY",40.726195,-74.004336,"Torch & Crown Brewing Company, in Hudson Squar...",https://www.torchandcrown.com,[],Manhattan,Hudson Square,Nightlife,Hudson Square,"['Week 2 (July 28 - Aug 3)', 'Week 3 (Aug 4 - ...",https://images.ctfassets.net/1aemqu6a6t65/3WPZ...,$30 Lunch; $45 Dinner; $30 Sunday Lunch/Brunch,American (New); Hudson Square
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
622,Catch n' Chop,https://nyc-tourism-public.s3.amazonaws.com/sr...,catch-n-chop,https://www.nyctourism.com/restaurant-week/cat...,"1600 Third Ave.,Manhattan,10128,NY",40.781370,-73.952445,"Catch n' Chop, situated on the Upper East Side...",https://catchnchop.com,[],Manhattan,Upper East Side,Restaurants,Upper East Side,"['Week 2 (July 28 - Aug 3)', 'Week 3 (Aug 4 - ...",NaN,$60 Dinner; $30 Sunday Lunch/Brunch; $60 Sunda...,Steakhouse; Upper East Side
623,Amélie - Upper West Side,,amelie-upper-west-side,https://www.nyctourism.com/restaurant-week/ame...,"566 Amsterdam Ave.,Manhattan,10024,NY",40.788751,-73.974624,"""La Petite Amélie"" is smaller and more intimat...",https://www.amelieupperwest.com,[],Manhattan,Upper West Side,Restaurants,Upper West Side,"['Week 2 (July 28 - Aug 3)', 'Week 3 (Aug 4 - ...",NaN,$45 Dinner; $45 Sunday Dinner,French; Upper West Side
624,Fogo de Chão Brazilian Steakhouse - Brooklyn,,fogo-de-chao-brazilian-steakhouse-brooklyn,https://www.nyctourism.com/restaurant-week/fog...,"445 Albee Sq. W.,Brooklyn,11201,NY",40.690724,-73.983285,This Downtown Brooklyn restaurant features an ...,https://fogodechao.com/location/brooklyn,[],Brooklyn,Downtown Brooklyn,Restaurants,Downtown Brooklyn,"['Week 2 (July 28 - Aug 3)', 'Week 3 (Aug 4 - ...",NaN,$60 Dinner; $60 Sunday Dinner,Brazilian; Downt

In [7]:
all_rest_data_df.to_excel("rw_restaurants_2025-07_full_v3.xlsx", index=False)